In [144]:
import pandas as pd
import plotly.express as px
import time

shs = pd.ExcelFile('RiskControl.xlsx').sheet_names
date_ = shs[-2]  #        日期修改在這裡!!!!!!!!!


df = pd.read_excel('RiskControl.xlsx', sheet_name=date_)
df["日期"] = date_

for broker in ["ADM","IFP","PHI","SCG"]:
    df1 = df[df['上手'].isin([broker]) & df['幣別'].isin(['USD'])]
    df_up5 = df1[df1["占比"]>=0.05]
    df_low5 = df1[df1["占比"]<0.05]
    new = pd.DataFrame.from_dict({"上手":[broker],
                                  "商品":['Other'],
                                  "月份":["Other"],
                                  "交易所":["Other"],
                                  "買方口數":[df_low5.買方口數.sum()],
                                  "賣方口數":[df_low5.賣方口數.sum()],
                                  "保證金":[0],
                                  "加總":[df_low5.加總.sum()],
                                  "結算價":['0'],
                                  "幣別":['USD'],
                                  "占比":[df_low5.占比.sum()],
                                  "調整後保證金":[df_low5.調整後保證金.sum()],
                                  "日期":[date_]})
    df_up5 = df_up5.append(new, ignore_index=True)
    df_all_up5 = df_up5 if broker == "ADM" else pd.concat([df_up5, df_all_up5], axis=0)

fig = px.sunburst(df_all_up5, path=["上手", "商品","月份"],
                  values='加總',width=1200, height=800,
                  color = '上手',
                  labels  = {"加總":"權益數"},
                  color_discrete_sequence=px.colors.qualitative.G10,
                  maxdepth = 3,
                  branchvalues='total',
                  title = shs[-1] + " 統一期貨美金保證金"
                  )

fig.update_traces(textinfo = 'label+percent entry+value+text',
                  insidetextfont=dict(size=23,family='Times New Roman')                
                  )
fig.update_layout(title_font=dict(size=23,family='Times New Roman'))
fig.show()